Follow instructions at the link below to setup NGSolve.

https://ngsolve.org/downloads

Code below assumes mesh has already been created using the 'mesh_creation.ipynb' script - creates a mesh of a simple sphere in a box. Here we do not distinguish between materials for the two regions.

## Simple Eddy Current Solver 

Conducting permable sphere $B$ with $\gamma = \gamma_* $, $\mu=\mu_*$ in $B$ and $\gamma=0$, $\mu=\mu_0$ in ${\mathbb R}^3 \backslash B$. Sphere placed in a uniform field H_0=[0 0 1]

We construct an ${\bf A}_0$ such that ${\bf B}_0 = \mu_0{\bf H}_0 =  \text{curl} {\bf A}_0$. This is the background field and the solution to

$$\text{curl} \mu_0^{-1} \text{curl} {\bf A}_0 = {\bf 0} \text{ in ${\mathbb R}^3$ }$$
$$\text{div} {\bf A}_0 = 0 \text{ in ${\mathbb R}^3$}$$

The full potential satisifies

$$\text{curl} \mu_0^{-1} \text{curl} {\bf A} = {\bf 0} \text{ in $ {\mathbb R}^3$ \ B }$$
$$\text{curl} \mu_*^{-1} \text{curl} {\bf A} + {\rm i}\omega  \gamma {\bf A} = {\bf 0} \text{ in $  B $}$$
$$\text{div} {\bf A} = 0 \text{ in ${\mathbb R}^3$}$$
$$[ {\bf n} \times {\bf A} ] = [ {\bf n} \times \mu^{-1} \text{curl}{\bf A}] = {\bf 0} \text{ on $\partial B$} $$ 
$$( {\bf A} - {\bf A}_0)({\bf x}) = O(1/|{\bf x}|) \text{ as $|{\bf x}| \to \infty$}$$ 

Multiply by $\mu_0$ and introduce $\mu_r = \mu_* / \mu_0$.

$$\text{curl}  \text{curl} {\bf A} = {\bf 0} \text{ in $ {\mathbb R}^3$ \ B }$$
$$\text{curl} \mu_r^{-1} \text{curl}  {\bf A} + {\rm i} \mu_0 \omega \gamma {\bf A}= {\bf 0} \text{ in $  B $}$$
$$\text{div} {\bf A} = 0 \text{ in ${\mathbb R}^3$}$$
$$[ {\bf n} \times  {\bf A} ] ={\bf 0},   {\bf n} \times \text{curl} {\bf A}|_+ -  {\bf n} \times \mu_r^{-1} \text{curl} {\bf A}|_- = {\bf 0}  \text{ on $\partial B$} $$ 
$$(  {\bf A}- {\bf A}_0 )({\bf x}) = O(1/|{\bf x}|) \text{ as $|{\bf x}| \to \infty$}$$ 


Weak form of the regularised problem on bounded domain $\Omega\subset {\mathbb R}^3$ is: Find $ {\bf A} \in X({\bf A}_0) = \{ {\bf a} \in H(\text{curl}) : \text{div} {\bf A} = 0 \text{ in $\Omega$}, {\bf n} \times {\bf A}= {\bf n } \times {\bf A}_0 \text{ on $\partial \Omega$}  \} $ such that

$$\int_{\Omega \backslash B} \text{curl} {\bf A} \cdot \text{curl} \overline{\delta {\bf A}} d \Omega+ \int_{ B} \mu_r^{-1} \text{curl} {\bf A} \cdot \text{curl} \overline{\delta {\bf A}} d \Omega +{\rm i} \mu_0 \omega \gamma_* \int_B {\bf A} \cdot \overline{\delta {\bf A}} d \Omega =
0$$
for all $ \delta {\bf A} \in X({\bf 0})$. Replace with regularised problem: $ {\bf A} \in X^\epsilon ({\bf A}_0) = \{ {\bf a} \in H(\text{curl}) : {\bf n} \times {\bf A}= {\bf n } \times {\bf A}_0  \text{ on $\partial \Omega$}  \} $

$$\int_{\Omega \backslash B} \text{curl} {\bf A}^\epsilon \cdot \text{curl} \overline{ \delta {\bf A}^\epsilon} d \Omega+ \int_{ B} \mu_r^{-1} \text{curl} {\bf A}^\epsilon \cdot \text{curl} \overline {\delta {\bf A}^\epsilon} d \Omega + \epsilon \int_\Omega {\bf A}^\epsilon \cdot \overline{\delta{\bf A}^\epsilon}  d \Omega +{\rm i} \mu_0 \omega \gamma_* \int_B {\bf A}^\epsilon \cdot \overline{\delta {\bf A}^\epsilon} d \Omega=
0$$
for all $ \delta {\bf A}^\epsilon \in X^\epsilon({\bf 0})$.


In [1]:
# Import libaries
import numpy as np
import netgen.meshing as ngmeshing
from ngsolve import *
import scipy.sparse as sp
import gc

# Specify order of elements
for Order in range(0,4):
    # Specify the mesh file
    Object = "OCC_sphere.vol"
    # Loading the object file
    ngmesh = ngmeshing.Mesh(dim=3)
    ngmesh.Load("VolFiles/" + Object)
    
    # Creating the mesh and defining the element types
    mesh = Mesh("VolFiles/" + Object)
    curve = 4
    mesh.Curve(curve)  # This can be used to set the degree of curved elements
    numelements = mesh.ne  # Count the number elements
    print(" mesh contains " + str(numelements) + " elements")

    # Materials consist of sphere material and air in the order as defined in the mesh
    matlist = ["copper", "air"]
    contrast = 20
    murlist = [contrast, 1 ]
    gammastar = 1e6
    gammalist = [gammastar,0]
    
    inout = []
    for mat in matlist:
        if mat == "air":
            inout.append(0)
        else:
            inout.append(1)
    inorout = dict(zip(matlist, inout))
    murmat = dict(zip(matlist, murlist))
    gammamat = dict(zip(matlist, gammalist))
    

    # Coefficient functions
    mur_coef = [murmat[mat] for mat in mesh.GetMaterials()]
    mur = CoefficientFunction(mur_coef)
    # inout = 1 in B, inout =0 in R^3 \ B
    inout_coef = [inorout[mat] for mat in mesh.GetMaterials()]
    inout = CoefficientFunction(inout_coef)
    gamma_coef = [gammamat[mat] for mat in mesh.GetMaterials()]
    gamma = CoefficientFunction(gamma_coef)
    

    # define material constants
    Mu0 = 4. * np.pi*1e-7
    Epsilon = 1e-6
    Omega = 10 # Frequency in rad/s
    
    # Coefficent function for background field
    B0 = 1
    H0 = CoefficientFunction((0,0,B0/Mu0))
    alpha = 1
    exactsphere = 1

    # define tolerances etc
    Maxsteps = 500
    Tolerance = 1e-8
    Solver = "bddc"

    print("Order=",Order)

    fes = HCurl(mesh, order=Order, dirichlet="outer", complex=True)

    # Count the number of degrees of freedom
    ndof = fes.ndof
    print("ndof",ndof)

    # Set up the grid function and apply Dirichlet BC
    a = GridFunction(fes)

    # Setup boundary condition
    if exactsphere==0:
        a.Set((0,0,0), BND)
    else:
        Mu = contrast * Mu0
        ka = np.sqrt(1j*gammastar*Omega*Mu)*alpha;
        k = np.sqrt(1j*gammastar*Omega*Mu)
        
        
        Ip12 = np.sqrt(2/pi/ka)*np.sinh(ka);
        Im12 = np.sqrt(2/pi/ka)*np.cosh(ka);
        
        # Equation Coefficient (Outside of sphere)
        numerator = ((2 * Mu + Mu0) * k * alpha * Im12 - (Mu0 * (1 + (k * alpha) ** 2) + 2 * Mu) * Ip12)
        denominator = ((Mu - Mu0) * k * alpha * Im12 + (Mu0 * (1 + (k * alpha) ** 2) - Mu) * Ip12)
        D = numerator/ denominator * alpha**3
        
        numerator = 3*Mu*k*alpha*alpha**(3/2)
        C = numerator/ denominator
        #print(D,C,numerator,denominator)
        def axout(x,y,z):    
            r=sqrt(x**2+y**2+z**2);
            rho = sqrt(x**2+y**2)
            theta=acos(z/r);
            phi=atan2(y,x);
            Aphi=1/2*B0*(r+D/r**2)*sin(theta)
            return -sin(phi)*Aphi
        def ayout(x,y,z):    
            r=sqrt(x**2+y**2+z**2);
            theta=acos(z/r);
            phi=atan2(y,x);
            Aphi=1/2*B0*(r+D/r**2)*sin(theta)
            return cos(phi)*Aphi
        def azout(x,y,z):    
            return 0.
        a.Set((axout(x,y,z), ayout(x,y,z), azout(x,y,z)), BND)

    # Setup source condition
    src = CoefficientFunction((0,0,0))

    # Test and trial functions
    u = fes.TrialFunction()
    v = fes.TestFunction()

    Additional_Int_Order = 2
    # Create the linear and bilinear forms (nb we get the linear system TSM.mat a.vec.data = - R = f.vec.data)
    f = LinearForm(fes)
    f += SymbolicLFI(  (src * v),bonus_intorder=Additional_Int_Order)

    TSM = BilinearForm(fes, symmetric=True, condense=True)
    TSM += SymbolicBFI(inout * (mur ** (-1)) * (curl(u) * curl(v)),bonus_intorder=Additional_Int_Order)
    TSM += SymbolicBFI((1-inout) * (curl(u) * curl(v)),bonus_intorder=Additional_Int_Order)
    TSM += SymbolicBFI(Epsilon* (1-inout) * (u * v),bonus_intorder=Additional_Int_Order)
    TSM += SymbolicBFI((-1)**(1/2) * inout * gamma * Omega * Mu0 * (u * v),bonus_intorder=Additional_Int_Order)
    

    if Solver == "bddc":
        P = Preconditioner(TSM, "bddc")  # Apply the bddc preconditioner
        print("using bddc")
    TSM.Assemble()
    f.Assemble()
    if Solver == "local":
        P= Preconditioner(TSM, "local")  # Apply the local preconditioner
    P.Update()

    # Solve the problem (including static condensation)
    f.vec.data += TSM.harmonic_extension_trans * f.vec
    res = f.vec.CreateVector()
    res.data = f.vec - (TSM.mat * a.vec)
    inverse = CGSolver(TSM.mat, P.mat, precision=Tolerance, maxsteps=Maxsteps, printrates=True)
    a.vec.data += inverse* res
    a.vec.data += TSM.harmonic_extension * a.vec

    a.vec.data += TSM.inner_solve * f.vec
    print("finished solve")

    # Compute errors
    # Compare with the exact solution
    # Construct it inside the object
    if exactsphere==1:
        Pi = np.pi
        def bxin(x,y,z):  
            r = sqrt(x**2+y**2+z**2);
            phi = atan2(y,x)
            theta = acos(z/r)
            v1 = k*r
            # Note that cosh and sinh are not available in the Coefficent function
            # They can not be called from numpy as argument is not a float or a complex value
            coshv1 = (exp(v1)+exp(-v1))/2
            sinhv1 = (exp(v1)-exp(-v1))/2
            I32 = sqrt(2/pi/v1)*(coshv1-1./v1*sinhv1) 
            Br = C*B0*cos(theta)*I32/r**(3/2)
            Ip12 = sqrt(2/Pi/v1)*sinhv1;
            Im12 = sqrt(2/Pi/v1)*coshv1;
            Btheta = -C/2*B0*sin(theta)/r**(3/2)*((v1+1/v1)*Ip12-Im12)
            return sin(theta)*cos(phi)*Br+cos(theta)*cos(phi)*Btheta
        def byin(x,y,z):
            r = sqrt(x**2+y**2+z**2);
            phi = atan2(y,x)
            theta = acos(z/r)
            v1 = k*r
            # Note that cosh and sinh are not available in the Coefficent function
            # They can not be called from numpy as argument is not a float or a complex value
            coshv1 = (exp(v1)+exp(-v1))/2
            sinhv1 = (exp(v1)-exp(-v1))/2
            I32 = sqrt(2/pi/v1)*(coshv1-1./v1*sinhv1) 
            Br = C*B0*cos(theta)*I32/r**(3/2)
            Ip12 = sqrt(2/Pi/v1)*sinhv1;
            Im12 = sqrt(2/Pi/v1)*coshv1;
            Btheta = -C/2*B0*sin(theta)/r**(3/2)*((v1+1/v1)*Ip12-Im12)
            return sin(theta)*sin(phi)*Br+cos(theta)*sin(phi)*Btheta
        def bzin(x,y,z):
            r = sqrt(x**2+y**2+z**2);
            phi = atan2(y,x)
            theta = acos(z/r)
            v1 = k*r
            # Note that cosh and sinh are not available in the Coefficent function
            # They can not be called from numpy as argument is not a float or a complex value
            coshv1 = (exp(v1)+exp(-v1))/2
            sinhv1 = (exp(v1)-exp(-v1))/2
            I32 = sqrt(2/pi/v1)*(coshv1-1./v1*sinhv1) 
            Br = C*B0*cos(theta)*I32/r**(3/2)
            Ip12 = sqrt(2/Pi/v1)*sinhv1;
            Im12 = sqrt(2/Pi/v1)*coshv1;
            Btheta = -C/2*B0*sin(theta)/r**(3/2)*((v1+1/v1)*Ip12-Im12)
            return cos(theta)*Br-sin(theta)*Btheta
        bexactinside = CoefficientFunction((bxin(x,y,z),byin(x,y,z),bzin(x,y,z)))
        
#       Do not use Bexact obtained from differentiating solution - has
#       precision issues. Recall that for a sphere (H_alpha - H_0 ) = D2G(x,z)M H_0 is
#       exact when H_0 is uniform
        #def logr(r):
        #    out=0.
        #    for n in range(20):
        #        out = out + 1/(1+2*n)*((r-1)/(r+1))**(n+1+2*n)
        #    return 2*out

        # Construct it outside the object
        #def bxout(x,y,z):
        #    r = sqrt(x**2+y**2+z**2);
        #    phi = atan2(y,x)
        #    theta = acos(z/r)
        #    Br = B0*(1-2*(1-contrast)/(2+contrast)*alpha**3/r**3)*cos(theta);
        #    #logr =  2 *( (r-1)/(r+1)  + (1/3)*( (r-1)/(r+1) )**3 + (1/5)* ( (r-1)/(r+1) )**5 + (1/7) *( (r-1)/(r+1) )**7 )
        #    Btheta = -B0*sin(theta)*(1-(1-contrast)/(2+contrast)*alpha**3*logr(r)/r);
        #    return sin(theta)*cos(phi)*Br+cos(theta)*cos(phi)*Btheta
               
        #def byout(x,y,z):
        #    r = sqrt(x**2+y**2+z**2);
        #    phi = atan2(y,x)
        #    theta = acos(z/r)
        #    Br = B0*(1-2*(1-contrast)/(2+contrast)*alpha**3/r**3)*cos(theta);
        #    #logr =  2 *( (r-1)/(r+1)  + (1/3)*( (r-1)/(r+1) )**3 + (1/5)* ( (r-1)/(r+1) )**5 + (1/7) *( (r-1)/(r+1) )**7 )
        #    Btheta = -B0*sin(theta)*(1-(1-contrast)/(2+contrast)*alpha**3*logr(r)/r);
        #    return sin(theta)*sin(phi)*Br+cos(theta)*sin(phi)*Btheta

        #def bzout(x,y,z):
        #    r = sqrt(x**2+y**2+z**2);
        #    phi = atan2(y,x)
        #    theta = acos(z/r)
        #    Br = B0*(1-2*(1-contrast)/(2+contrast)*alpha**3/r**3)*cos(theta);
        #    #logr =  2 *( (r-1)/(r+1)  + (1/3)*( (r-1)/(r+1) )**3 + (1/5)* ( (r-1)/(r+1) )**5 + (1/7) *( (r-1)/(r+1) )**7 )
        #    Btheta = -B0*sin(theta)*(1-(1-contrast)/(2+contrast)*alpha**3*logr(r)/r);
        #    return cos(theta)*Br-sin(theta)*Btheta
 
        Pi=np.pi
        def Polarisation():
            
            Mu = contrast * Mu0
            k = np.sqrt( Mu * gammastar * Omega * 1j)

            Ip12 = np.sqrt(2 / Pi / (k * alpha)) * np.sinh(k * alpha)
            Im12 = np.sqrt(2 / Pi / (k * alpha)) * np.cosh(k * alpha)

        # sinh and cosh return inf for large values of k*alpha. This leads to the numerator and denominator both being inf.
        # inf/inf is undefined so eig = nan.
            numerator = ((2 * Mu + Mu0) * k * alpha * Im12 - (Mu0 * (1 + (k * alpha) ** 2) + 2 * Mu) * Ip12)
            denominator = ((Mu - Mu0) * k * alpha * Im12 + (Mu0 * (1 + (k * alpha) ** 2) - Mu) * Ip12)
            return 2 * Pi * alpha ** 3 * numerator / denominator
        M = Polarisation()
        #print(M)
    
        def bxout(x,y,z):
            r = sqrt(x**2+y**2+z**2)
            rx=x/r
            ry=y/r
            rz=z/r
            #M=4*Pi*alpha**3*(contrast-1)/(2+contrast)
            return M/(4*Pi*r**3)*3*rx*rz*B0
        
        def byout(x,y,z):
            r = sqrt(x**2+y**2+z**2)
            rx=x/r
            ry=y/r
            rz=z/r
            #M=4*Pi*alpha**3*(contrast-1)/(2+contrast)
            return M/(4*Pi*r**3)*3*ry*rz*B0
        
        def bzout(x,y,z):
            r = sqrt(x**2+y**2+z**2)
            rx=x/r
            ry=y/r
            rz=z/r
            #M=4*Pi*alpha**3*(contrast-1)/(2+contrast)
            return M/(4*Pi*r**3)*(3*rz*rz-1)*B0+B0
             
            
        bexactoutside = CoefficientFunction((bxout(x,y,z),byout(x,y,z),bzout(x,y,z)))
        
        
        # Modifed Bessel function of second kind Taylor expand
        #Ip32 = sqrt(2/pi/v1)*(cosh(v1)-1./v1*sinh(v1))        
        #def Ip32(r):
        #    return r**(3/2)/3*sqrt(2/Pi)+r**(7/2)/(15*sqrt(2*pi))+r**(11/2)/(420*sqrt(2*Pi))
        # Aphi inside the sphere depends on a spherical bessel function and on
        # hyperbolic functions not available in the standard Coefficent function
        # One option may be this https://github.com/NGSolve/ngs-special-functions
        # Implementation below is expected to be inaccurate for high omega.
        def axin(x,y,z):    
            r=sqrt(x**2+y**2+z**2)
            theta=acos(z/r)
            phi=atan2(y,x)
            v1 = k*r
            # Note that cosh and sinh are not available in the Coefficent function
            # They can not be called from numpy as argument is not a float or a complex value
            coshv1 = (exp(v1)+exp(-v1))/2
            sinhv1 = (exp(v1)-exp(-v1))/2
            I32 = sqrt(2/pi/v1)*(coshv1-1./v1*sinhv1) 
            Aphi=B0/2*C/r**(1/2)*I32*sin(theta)
            return -sin(phi)*Aphi
        def ayin(x,y,z):    
            r=sqrt(x**2+y**2+z**2)
            theta=acos(z/r)
            phi=atan2(y,x)
            v1 = k*r
            coshv1 = (exp(v1)+exp(-v1))/2
            sinhv1 = (exp(v1)-exp(-v1))/2
            I32 = sqrt(2/pi/v1)*(coshv1-1./v1*sinhv1) 
            Aphi=B0/2*C/r**(1/2)*I32*sin(theta)
            return cos(phi)*Aphi
        def azin(x,y,z):    
            return 0
        aexactoutside = CoefficientFunction((axout(x,y,z),ayout(x,y,z),azout(x,y,z)))
        aexactinside = CoefficientFunction((axin(x,y,z),ayin(x,y,z),azin(x,y,z)))


    # Compute L2 norm of curl error = curl (a-aexact) = curl (a -aexact) = curl (a)  -bexact
        Integration_Order = np.max([4*(Order+1),3*(curve-1)])

        Ierrinside = Integrate(inout*InnerProduct(curl(a)-bexactinside,curl(a)-bexactinside),mesh,order=Integration_Order)
        Ierroutside = Integrate((1-inout)*InnerProduct(curl(a)-bexactoutside,curl(a)-bexactoutside),mesh,order=Integration_Order)
        Ierrtot = Ierroutside+Ierrinside
        Ininside = Integrate(inout*InnerProduct(bexactinside,bexactinside),mesh,order=Integration_Order)
        Inoutside = Integrate((1-inout)*InnerProduct(bexactoutside,bexactoutside),mesh,order=Integration_Order)
        Intot = Inoutside+Ininside
        print("Order=",Order,"Error in curl",np.real(Ierrtot/Intot))
        print("Order=",Order,"Error in curl inside",np.real(Ierrinside/Ininside))
        print("Order=",Order,"Error in curl outside",np.real(Ierroutside/Inoutside))
                
    
    
        Ierrinside = Integrate(inout*InnerProduct(a-aexactinside,a-aexactinside),mesh,order=Integration_Order)
        Ierroutside = Integrate((1-inout)*InnerProduct(a-aexactoutside,a-aexactoutside),mesh,order=Integration_Order)
        Ierrtot = Ierrinside+Ierroutside
        Ininside = Integrate(inout*InnerProduct(aexactinside,aexactinside),mesh,order=Integration_Order)
        Inoutside = Integrate((1-inout)*InnerProduct(aexactoutside,aexactoutside),mesh,order=Integration_Order)
        Intot = Ininside+Inoutside
        print("Order=",Order,"Error in a",np.real(Ierrtot/Intot))
        print("Order=",Order,"Error in a outside",np.real(Ierroutside/Inoutside))
        print("Order=",Order,"Error in a inside",np.real(Ierrinside/Ininside))
        


importing NGSolve-6.2.2204
 mesh contains 12196 elements
Order= 0
ndof 14409
using bddc
finished solve
Order= 0 Error in curl 0.003935653596862793
Order= 0 Error in curl inside 0.40607593387269586
Order= 0 Error in curl outside 6.173052314789067e-05
Order= 0 Error in a 4.042228819685064e-06
Order= 0 Error in a outside 3.004216555306521e-06
Order= 0 Error in a inside 0.4560221461750417
 mesh contains 12196 elements
Order= 1
ndof 28818
using bddc
finished solve
Order= 1 Error in curl 0.0038528234868653905
Order= 1 Error in curl inside 0.4008249745651167
Order= 1 Error in curl outside 2.8686360812133124e-05
Order= 1 Error in a 1.1118420151730496e-06
Order= 1 Error in a outside 7.072194893360929e-07
Order= 1 Error in a inside 0.17775931699826084
 mesh contains 12196 elements
Order= 2
ndof 116637
using bddc
finished solve
Order= 2 Error in curl 0.0005566861428567304
Order= 2 Error in curl inside 0.05828479471852361
Order= 2 Error in curl outside 5.758370575056095e-07
Order= 2 Error in a 4.7

Now try with gradient removed in the region outside. Those in the conductor must be retained.

In [2]:
# Import libaries
import numpy as np
import netgen.meshing as ngmeshing
from ngsolve import *
import scipy.sparse as sp
import gc

# Specify order of elements
for Order in range(0,4):
    # Specify the mesh file
    Object = "OCC_sphere.vol"
    # Loading the object file
    ngmesh = ngmeshing.Mesh(dim=3)
    ngmesh.Load("VolFiles/" + Object)
    
    # Creating the mesh and defining the element types
    mesh = Mesh("VolFiles/" + Object)
    curve = 4
    mesh.Curve(curve)  # This can be used to set the degree of curved elements
    numelements = mesh.ne  # Count the number elements
    print(" mesh contains " + str(numelements) + " elements")

    # Materials consist of sphere material and air in the order as defined in the mesh
    matlist = ["copper", "air"]
    contrast = 20
    murlist = [contrast, 1 ]
    gammastar = 1e6
    gammalist = [gammastar,0]
    
    inout = []
    for mat in matlist:
        if mat == "air":
            inout.append(0)
        else:
            inout.append(1)
    inorout = dict(zip(matlist, inout))
    murmat = dict(zip(matlist, murlist))
    gammamat = dict(zip(matlist, gammalist))
    

    # Coefficient functions
    mur_coef = [murmat[mat] for mat in mesh.GetMaterials()]
    mur = CoefficientFunction(mur_coef)
    # inout = 1 in B, inout =0 in R^3 \ B
    inout_coef = [inorout[mat] for mat in mesh.GetMaterials()]
    inout = CoefficientFunction(inout_coef)
    gamma_coef = [gammamat[mat] for mat in mesh.GetMaterials()]
    gamma = CoefficientFunction(gamma_coef)
    

    # define material constants
    Mu0 = 4. * np.pi*1e-7
    Epsilon = 1e-6
    Omega = 10 # Frequency in rad/s
    
    # Coefficent function for background field
    B0 = 1
    H0 = CoefficientFunction((0,0,B0/Mu0))
    alpha = 1
    exactsphere = 1

    # define tolerances etc
    Maxsteps = 500
    Tolerance = 1e-8
    Solver = "bddc"

    print("Order=",Order)

    dom_nrs_metal = [0 if mat == "air" else 1 for mat in mesh.GetMaterials()]
    fes = HCurl(mesh, order=Order, dirichlet="outer", complex=True, gradientdomains=dom_nrs_metal)

    # Count the number of degrees of freedom
    ndof = fes.ndof
    print("ndof",ndof)

    # Set up the grid function and apply Dirichlet BC
    a = GridFunction(fes)

    # Setup boundary condition
    if exactsphere==0:
        a.Set((0,0,0), BND)
    else:
        Mu = contrast * Mu0
        ka = np.sqrt(1j*gammastar*Omega*Mu)*alpha;
        k = np.sqrt(1j*gammastar*Omega*Mu)
        
        
        Ip12 = np.sqrt(2/pi/ka)*np.sinh(ka);
        Im12 = np.sqrt(2/pi/ka)*np.cosh(ka);
        
        # Equation Coefficient (Outside of sphere)
        numerator = ((2 * Mu + Mu0) * k * alpha * Im12 - (Mu0 * (1 + (k * alpha) ** 2) + 2 * Mu) * Ip12)
        denominator = ((Mu - Mu0) * k * alpha * Im12 + (Mu0 * (1 + (k * alpha) ** 2) - Mu) * Ip12)
        D = numerator/ denominator * alpha**3
        
        numerator = 3*Mu*k*alpha*alpha**(3/2)
        C = numerator/ denominator
        #print(D,C,numerator,denominator)
        def axout(x,y,z):    
            r=sqrt(x**2+y**2+z**2);
            rho = sqrt(x**2+y**2)
            theta=acos(z/r);
            phi=atan2(y,x);
            Aphi=1/2*B0*(r+D/r**2)*sin(theta)
            return -sin(phi)*Aphi
        def ayout(x,y,z):    
            r=sqrt(x**2+y**2+z**2);
            theta=acos(z/r);
            phi=atan2(y,x);
            Aphi=1/2*B0*(r+D/r**2)*sin(theta)
            return cos(phi)*Aphi
        def azout(x,y,z):    
            return 0.
        a.Set((axout(x,y,z), ayout(x,y,z), azout(x,y,z)), BND)

    # Setup source condition
    src = CoefficientFunction((0,0,0))

    # Test and trial functions
    u = fes.TrialFunction()
    v = fes.TestFunction()

    Additional_Int_Order = 2
    # Create the linear and bilinear forms (nb we get the linear system TSM.mat a.vec.data = - R = f.vec.data)
    f = LinearForm(fes)
    f += SymbolicLFI(  (src * v),bonus_intorder=Additional_Int_Order)

    TSM = BilinearForm(fes, symmetric=True, condense=True)
    TSM += SymbolicBFI(inout * (mur ** (-1)) * (curl(u) * curl(v)),bonus_intorder=Additional_Int_Order)
    TSM += SymbolicBFI((1-inout) * (curl(u) * curl(v)),bonus_intorder=Additional_Int_Order)
    TSM += SymbolicBFI(Epsilon* (1-inout) * (u * v),bonus_intorder=Additional_Int_Order)
    TSM += SymbolicBFI((-1)**(1/2) * inout * gamma * Omega * Mu0 * (u * v),bonus_intorder=Additional_Int_Order)
    

    if Solver == "bddc":
        P = Preconditioner(TSM, "bddc")  # Apply the bddc preconditioner
        print("using bddc")
    TSM.Assemble()
    f.Assemble()
    if Solver == "local":
        P= Preconditioner(TSM, "local")  # Apply the local preconditioner
    P.Update()

    # Solve the problem (including static condensation)
    f.vec.data += TSM.harmonic_extension_trans * f.vec
    res = f.vec.CreateVector()
    res.data = f.vec - (TSM.mat * a.vec)
    inverse = CGSolver(TSM.mat, P.mat, precision=Tolerance, maxsteps=Maxsteps, printrates=True)
    a.vec.data += inverse* res
    a.vec.data += TSM.harmonic_extension * a.vec

    a.vec.data += TSM.inner_solve * f.vec
    print("finished solve")

    # Compute errors
    # Compare with the exact solution
    # Construct it inside the object
    if exactsphere==1:
        Pi = np.pi
        def bxin(x,y,z):  
            r = sqrt(x**2+y**2+z**2);
            phi = atan2(y,x)
            theta = acos(z/r)
            v1 = k*r
            # Note that cosh and sinh are not available in the Coefficent function
            # They can not be called from numpy as argument is not a float or a complex value
            coshv1 = (exp(v1)+exp(-v1))/2
            sinhv1 = (exp(v1)-exp(-v1))/2
            I32 = sqrt(2/pi/v1)*(coshv1-1./v1*sinhv1) 
            Br = C*B0*cos(theta)*I32/r**(3/2)
            Ip12 = sqrt(2/Pi/v1)*sinhv1;
            Im12 = sqrt(2/Pi/v1)*coshv1;
            Btheta = -C/2*B0*sin(theta)/r**(3/2)*((v1+1/v1)*Ip12-Im12)
            return sin(theta)*cos(phi)*Br+cos(theta)*cos(phi)*Btheta
        def byin(x,y,z):
            r = sqrt(x**2+y**2+z**2);
            phi = atan2(y,x)
            theta = acos(z/r)
            v1 = k*r
            # Note that cosh and sinh are not available in the Coefficent function
            # They can not be called from numpy as argument is not a float or a complex value
            coshv1 = (exp(v1)+exp(-v1))/2
            sinhv1 = (exp(v1)-exp(-v1))/2
            I32 = sqrt(2/pi/v1)*(coshv1-1./v1*sinhv1) 
            Br = C*B0*cos(theta)*I32/r**(3/2)
            Ip12 = sqrt(2/Pi/v1)*sinhv1;
            Im12 = sqrt(2/Pi/v1)*coshv1;
            Btheta = -C/2*B0*sin(theta)/r**(3/2)*((v1+1/v1)*Ip12-Im12)
            return sin(theta)*sin(phi)*Br+cos(theta)*sin(phi)*Btheta
        def bzin(x,y,z):
            r = sqrt(x**2+y**2+z**2);
            phi = atan2(y,x)
            theta = acos(z/r)
            v1 = k*r
            # Note that cosh and sinh are not available in the Coefficent function
            # They can not be called from numpy as argument is not a float or a complex value
            coshv1 = (exp(v1)+exp(-v1))/2
            sinhv1 = (exp(v1)-exp(-v1))/2
            I32 = sqrt(2/pi/v1)*(coshv1-1./v1*sinhv1) 
            Br = C*B0*cos(theta)*I32/r**(3/2)
            Ip12 = sqrt(2/Pi/v1)*sinhv1;
            Im12 = sqrt(2/Pi/v1)*coshv1;
            Btheta = -C/2*B0*sin(theta)/r**(3/2)*((v1+1/v1)*Ip12-Im12)
            return cos(theta)*Br-sin(theta)*Btheta
        bexactinside = CoefficientFunction((bxin(x,y,z),byin(x,y,z),bzin(x,y,z)))
        
#       Do not use Bexact obtained from differentiating solution - has
#       precision issues. Recall that for a sphere (H_alpha - H_0 ) = D2G(x,z)M H_0 is
#       exact when H_0 is uniform
        #def logr(r):
        #    out=0.
        #    for n in range(20):
        #        out = out + 1/(1+2*n)*((r-1)/(r+1))**(n+1+2*n)
        #    return 2*out

        # Construct it outside the object
        #def bxout(x,y,z):
        #    r = sqrt(x**2+y**2+z**2);
        #    phi = atan2(y,x)
        #    theta = acos(z/r)
        #    Br = B0*(1-2*(1-contrast)/(2+contrast)*alpha**3/r**3)*cos(theta);
        #    #logr =  2 *( (r-1)/(r+1)  + (1/3)*( (r-1)/(r+1) )**3 + (1/5)* ( (r-1)/(r+1) )**5 + (1/7) *( (r-1)/(r+1) )**7 )
        #    Btheta = -B0*sin(theta)*(1-(1-contrast)/(2+contrast)*alpha**3*logr(r)/r);
        #    return sin(theta)*cos(phi)*Br+cos(theta)*cos(phi)*Btheta
               
        #def byout(x,y,z):
        #    r = sqrt(x**2+y**2+z**2);
        #    phi = atan2(y,x)
        #    theta = acos(z/r)
        #    Br = B0*(1-2*(1-contrast)/(2+contrast)*alpha**3/r**3)*cos(theta);
        #    #logr =  2 *( (r-1)/(r+1)  + (1/3)*( (r-1)/(r+1) )**3 + (1/5)* ( (r-1)/(r+1) )**5 + (1/7) *( (r-1)/(r+1) )**7 )
        #    Btheta = -B0*sin(theta)*(1-(1-contrast)/(2+contrast)*alpha**3*logr(r)/r);
        #    return sin(theta)*sin(phi)*Br+cos(theta)*sin(phi)*Btheta

        #def bzout(x,y,z):
        #    r = sqrt(x**2+y**2+z**2);
        #    phi = atan2(y,x)
        #    theta = acos(z/r)
        #    Br = B0*(1-2*(1-contrast)/(2+contrast)*alpha**3/r**3)*cos(theta);
        #    #logr =  2 *( (r-1)/(r+1)  + (1/3)*( (r-1)/(r+1) )**3 + (1/5)* ( (r-1)/(r+1) )**5 + (1/7) *( (r-1)/(r+1) )**7 )
        #    Btheta = -B0*sin(theta)*(1-(1-contrast)/(2+contrast)*alpha**3*logr(r)/r);
        #    return cos(theta)*Br-sin(theta)*Btheta
 
        Pi=np.pi
        def Polarisation():
            
            Mu = contrast * Mu0
            k = np.sqrt( Mu * gammastar * Omega * 1j)

            Ip12 = np.sqrt(2 / Pi / (k * alpha)) * np.sinh(k * alpha)
            Im12 = np.sqrt(2 / Pi / (k * alpha)) * np.cosh(k * alpha)

        # sinh and cosh return inf for large values of k*alpha. This leads to the numerator and denominator both being inf.
        # inf/inf is undefined so eig = nan.
            numerator = ((2 * Mu + Mu0) * k * alpha * Im12 - (Mu0 * (1 + (k * alpha) ** 2) + 2 * Mu) * Ip12)
            denominator = ((Mu - Mu0) * k * alpha * Im12 + (Mu0 * (1 + (k * alpha) ** 2) - Mu) * Ip12)
            return 2 * Pi * alpha ** 3 * numerator / denominator
        M = Polarisation()
        #print(M)
    
        def bxout(x,y,z):
            r = sqrt(x**2+y**2+z**2)
            rx=x/r
            ry=y/r
            rz=z/r
            #M=4*Pi*alpha**3*(contrast-1)/(2+contrast)
            return M/(4*Pi*r**3)*3*rx*rz*B0
        
        def byout(x,y,z):
            r = sqrt(x**2+y**2+z**2)
            rx=x/r
            ry=y/r
            rz=z/r
            #M=4*Pi*alpha**3*(contrast-1)/(2+contrast)
            return M/(4*Pi*r**3)*3*ry*rz*B0
        
        def bzout(x,y,z):
            r = sqrt(x**2+y**2+z**2)
            rx=x/r
            ry=y/r
            rz=z/r
            #M=4*Pi*alpha**3*(contrast-1)/(2+contrast)
            return M/(4*Pi*r**3)*(3*rz*rz-1)*B0+B0
             
            
        bexactoutside = CoefficientFunction((bxout(x,y,z),byout(x,y,z),bzout(x,y,z)))
        
        
        # Modifed Bessel function of second kind Taylor expand
        #Ip32 = sqrt(2/pi/v1)*(cosh(v1)-1./v1*sinh(v1))        
        #def Ip32(r):
        #    return r**(3/2)/3*sqrt(2/Pi)+r**(7/2)/(15*sqrt(2*pi))+r**(11/2)/(420*sqrt(2*Pi))
        # Aphi inside the sphere depends on a spherical bessel function and on
        # hyperbolic functions not available in the standard Coefficent function
        # One option may be this https://github.com/NGSolve/ngs-special-functions
        # Implementation below is expected to be inaccurate for high omega.
        def axin(x,y,z):    
            r=sqrt(x**2+y**2+z**2)
            theta=acos(z/r)
            phi=atan2(y,x)
            v1 = k*r
            # Note that cosh and sinh are not available in the Coefficent function
            # They can not be called from numpy as argument is not a float or a complex value
            coshv1 = (exp(v1)+exp(-v1))/2
            sinhv1 = (exp(v1)-exp(-v1))/2
            I32 = sqrt(2/pi/v1)*(coshv1-1./v1*sinhv1) 
            Aphi=B0/2*C/r**(1/2)*I32*sin(theta)
            return -sin(phi)*Aphi
        def ayin(x,y,z):    
            r=sqrt(x**2+y**2+z**2)
            theta=acos(z/r)
            phi=atan2(y,x)
            v1 = k*r
            coshv1 = (exp(v1)+exp(-v1))/2
            sinhv1 = (exp(v1)-exp(-v1))/2
            I32 = sqrt(2/pi/v1)*(coshv1-1./v1*sinhv1) 
            Aphi=B0/2*C/r**(1/2)*I32*sin(theta)
            return cos(phi)*Aphi
        def azin(x,y,z):    
            return 0
        aexactoutside = CoefficientFunction((axout(x,y,z),ayout(x,y,z),azout(x,y,z)))
        aexactinside = CoefficientFunction((axin(x,y,z),ayin(x,y,z),azin(x,y,z)))


    # Compute L2 norm of curl error = curl (a-aexact) = curl (a -aexact) = curl (a)  -bexact
        Integration_Order = np.max([4*(Order+1),3*(curve-1)])

        Ierrinside = Integrate(inout*InnerProduct(curl(a)-bexactinside,curl(a)-bexactinside),mesh,order=Integration_Order)
        Ierroutside = Integrate((1-inout)*InnerProduct(curl(a)-bexactoutside,curl(a)-bexactoutside),mesh,order=Integration_Order)
        Ierrtot = Ierroutside+Ierrinside
        Ininside = Integrate(inout*InnerProduct(bexactinside,bexactinside),mesh,order=Integration_Order)
        Inoutside = Integrate((1-inout)*InnerProduct(bexactoutside,bexactoutside),mesh,order=Integration_Order)
        Intot = Inoutside+Ininside
        print("Order=",Order,"Error in curl",np.real(Ierrtot/Intot))
        print("Order=",Order,"Error in curl inside",np.real(Ierrinside/Ininside))
        print("Order=",Order,"Error in curl outside",np.real(Ierroutside/Inoutside))
                
    
    
        Ierrinside = Integrate(inout*InnerProduct(a-aexactinside,a-aexactinside),mesh,order=Integration_Order)
        Ierroutside = Integrate((1-inout)*InnerProduct(a-aexactoutside,a-aexactoutside),mesh,order=Integration_Order)
        Ierrtot = Ierrinside+Ierroutside
        Ininside = Integrate(inout*InnerProduct(aexactinside,aexactinside),mesh,order=Integration_Order)
        Inoutside = Integrate((1-inout)*InnerProduct(aexactoutside,aexactoutside),mesh,order=Integration_Order)
        Intot = Ininside+Inoutside
        print("Order=",Order,"Error in a",np.real(Ierrtot/Intot))
        print("Order=",Order,"Error in a outside",np.real(Ierroutside/Inoutside))
        print("Order=",Order,"Error in a inside",np.real(Ierrinside/Ininside))


 mesh contains 12196 elements
Order= 0
ndof 14409
using bddc
finished solve
Order= 0 Error in curl 0.003935653596862794
Order= 0 Error in curl inside 0.406075933872696
Order= 0 Error in curl outside 6.17305231478906e-05
Order= 0 Error in a 4.042228819685139e-06
Order= 0 Error in a outside 3.0042165553065967e-06
Order= 0 Error in a inside 0.45602214617504117
 mesh contains 12196 elements
Order= 1
ndof 17876
using bddc
finished solve
Order= 1 Error in curl 0.0038528234994695773
Order= 1 Error in curl inside 0.40082497597582983
Order= 1 Error in curl outside 2.8686359947968556e-05
Order= 1 Error in a 1.6763504210142668e-06
Order= 1 Error in a outside 1.2717291856851602e-06
Order= 1 Error in a inside 0.17775931456255326
 mesh contains 12196 elements
Order= 2
ndof 75387
using bddc
finished solve
Order= 2 Error in curl 0.0005566861424536509
Order= 2 Error in curl inside 0.05828479472942229
Order= 2 Error in curl outside 5.758365455531862e-07
Order= 2 Error in a 5.311061178008769e-07
Order= 2